In [35]:
import pandas as pd
import numpy as np
import os

In [36]:
os.getcwd()

'/home/akarshsahu/bigdata/Final_Project'

In [ ]:
## Title Basic Information

In [37]:
imdb_df = pd.read_csv("./IMDB_Data/title_basics.tsv", sep="\t")
title = pd.read_csv("./Netflix_Data/movie_titles.csv", encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])

/software/Anaconda3-5.1.0-hadoop/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
imdb_df.startYear = pd.to_numeric(imdb_df.startYear, errors = 'coerce')
imdb_df = imdb_df[imdb_df.titleType == 'movie']

In [40]:
imdb_df.titleType.unique()

array(['movie'], dtype=object)

In [41]:
# 17770 items in total
len(title)

17770

In [42]:
title.head()

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [43]:
merge_df = pd.merge(title, imdb_df[['tconst', 'primaryTitle', 'titleType', 'runtimeMinutes', 'startYear', 'isAdult', 'genres']], left_on=['Name', 'Year'], right_on = ['primaryTitle', 'startYear'], how='inner')

In [48]:
merge_df.head()

,Movie_Id,Year,Name,tconst,primaryTitle,titleType,runtimeMinutes,startYear,isAdult,genres
0,3,1997.0,Character,tt0119448,Character,movie,122,1997.0,0,"Crime,Drama,Mystery"
1,7,1992.0,8 Man,tt0182668,8 Man,movie,83,1992.0,0,"Action,Sci-Fi"
2,12,1947.0,My Favorite Brunette,tt0039645,My Favorite Brunette,movie,87,1947.0,0,"Comedy,Crime,Film-Noir"
3,18,1994.0,Immortal Beloved,tt0110116,Immortal Beloved,movie,121,1994.0,0,"Biography,Drama,Music"
4,20,1972.0,Seeta Aur Geeta,tt0078222,Seeta Aur Geeta,movie,162,1972.0,0,"Comedy,Drama,Family"


In [45]:
# Only keep the observations with max runtime for duplicate movies
imdb_temp = merge_df[['Movie_Id', 'tconst', 'runtimeMinutes']]
idx = imdb_temp.groupby('Movie_Id')['runtimeMinutes'].transform(max) == imdb_temp['runtimeMinutes']

imdb_temp = imdb_temp[idx]


In [60]:
merge_df.head()

,Movie_Id,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,isAdult,genres
0,3,tt0119448,1997.0,Character,Character,movie,122,1997.0,0,"Crime,Drama,Mystery"
1,7,tt0182668,1992.0,8 Man,8 Man,movie,83,1992.0,0,"Action,Sci-Fi"
2,12,tt0039645,1947.0,My Favorite Brunette,My Favorite Brunette,movie,87,1947.0,0,"Comedy,Crime,Film-Noir"
3,18,tt0110116,1994.0,Immortal Beloved,Immortal Beloved,movie,121,1994.0,0,"Biography,Drama,Music"
4,20,tt0078222,1972.0,Seeta Aur Geeta,Seeta Aur Geeta,movie,162,1972.0,0,"Comedy,Drama,Family"


In [50]:
merge_df = pd.merge(imdb_temp[['Movie_Id','tconst']], merge_df, on = ['tconst', 'Movie_Id'], how = "left")

In [57]:
merge_df.tconst.nunique()

7597

In [58]:
merge_df.Movie_Id.nunique()

7602

In [53]:
cnt = pd.DataFrame(merge_df.Movie_Id.value_counts()).reset_index()
cnt.columns = ['Movie_Id', 'cnt']

In [54]:
cnt_lst = pd.Series(cnt[cnt.cnt > 1].Movie_Id.unique())

In [141]:
# No Duplicate movie titles
cnt_lst

Series([], dtype: int64)

In [ ]:
## Ratings

In [61]:
imdb_df = pd.read_csv("./IMDB_Data/title_ratings.tsv", sep="\t")

imdb_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1586
1,tt0000002,6.1,191
2,tt0000003,6.5,1249
3,tt0000004,6.2,118
4,tt0000005,6.1,1998


In [62]:
merge_df = pd.merge(merge_df, imdb_df[['tconst', 'averageRating', 'numVotes']], left_on='tconst', right_on = 'tconst', how='left')

merge_df.head()

merge_df.Movie_Id.nunique()

7602

In [63]:
len(merge_df.tconst.unique())

7597

In [ ]:
## Crew Names

In [95]:
imdb_df = pd.read_csv("./IMDB_Data/title_principals.tsv", sep="\t")
imdb_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [96]:
imdb_df.category.unique()

array(['self', 'director', 'cinematographer', 'composer', 'producer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [97]:
imdb_df = imdb_df[imdb_df.category.isin(['director', 'actor', 'actress']) & imdb_df.tconst.isin(list(merge_df.tconst.unique()))]

In [98]:
imdb_df.category.unique()

array(['actress', 'actor', 'director'], dtype=object)

In [99]:
merge_df.tconst.nunique()

7597

In [100]:
imdb_df.tconst.nunique()

7570

In [101]:
imdb_df_name = pd.read_csv("./IMDB_Data/name_basics.tsv", sep="\t")

In [102]:
imdb_df_name.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0043044,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0071877,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0059956,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0077975,tt0080455,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"


In [103]:
imdb_df = pd.merge(imdb_df[['tconst', 'nconst', 'category']], imdb_df_name[['nconst', 'primaryName']], left_on='nconst', right_on = 'nconst', how = 'inner')

In [104]:
imdb_df.head()

,tconst,nconst,category,primaryName
0,tt0003740,nm0021935,actress,Italia Almirante-Manzini
1,tt0003740,nm0702894,actress,Lidia Quaranta
2,tt0003740,nm0656034,actor,Bartolomeo Pagano
3,tt0003740,nm0146028,actress,Carolina Catena
4,tt0003740,nm0665163,director,Giovanni Pastrone


In [105]:
imdb_df_dir = imdb_df[imdb_df.category == 'director']
imdb_df_act = imdb_df[imdb_df.category != 'director']
del imdb_df

In [106]:
imdb_df_dir.head()

,tconst,nconst,category,primaryName
4,tt0003740,nm0665163,director,Giovanni Pastrone
14,tt0061523,nm0000122,director,Charles Chaplin
20,tt0004707,nm0784407,director,Mack Sennett
21,tt0004707,nm0071658,director,Charles Bennett
32,tt0004972,nm0000428,director,D.W. Griffith


In [107]:
imdb_df_act.head()

,tconst,nconst,category,primaryName
0,tt0003740,nm0021935,actress,Italia Almirante-Manzini
1,tt0003740,nm0702894,actress,Lidia Quaranta
2,tt0003740,nm0656034,actor,Bartolomeo Pagano
3,tt0003740,nm0146028,actress,Carolina Catena
5,tt0004707,nm0000122,actor,Charles Chaplin


In [108]:
imdb_df_dir = imdb_df_dir.groupby(['tconst'])['primaryName'].apply(list).apply(pd.Series).reset_index()

In [110]:
imdb_df_dir.head()
#imdb_df_dir.to_csv("./IMDB_Data/imdb_df_dir.csv")

,tconst,0,1,2,3,4,5
0,tt0003740,Giovanni Pastrone,NaN,NaN,NaN,NaN,NaN
1,tt0004707,Mack Sennett,Charles Bennett,NaN,NaN,NaN,NaN
2,tt0004972,D.W. Griffith,NaN,NaN,NaN,NaN,NaN
3,tt0010418,Cecil B. DeMille,NaN,NaN,NaN,NaN,NaN
4,tt0011130,John S. Robertson,NaN,NaN,NaN,NaN,NaN


In [111]:
imdb_df_act = imdb_df_act.groupby(['tconst'])['primaryName'].apply(list).apply(pd.Series)

In [112]:
imdb_df_act = imdb_df_act.reset_index()
imdb_df_act.head()

,tconst,0,1,2,3,4,5,6,7,8,9
0,tt0003740,Italia Almirante-Manzini,Lidia Quaranta,Bartolomeo Pagano,Carolina Catena,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0004707,Charles Chaplin,Marie Dressler,Mabel Normand,Mack Swain,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0004972,Lillian Gish,Mae Marsh,Henry B. Walthall,Miriam Cooper,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0010418,Thomas Meighan,Theodore Roberts,Raymond Hatton,Robert Cain,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0011130,John Barrymore,Martha Mansfield,Brandon Hurst,Charles Lane,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
merge_df.head()

,Movie_Id,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,isAdult,genres,averageRating,numVotes
0,3,tt0119448,1997.0,Character,Character,movie,122,1997.0,0,"Crime,Drama,Mystery",7.7,10128.0
1,7,tt0182668,1992.0,8 Man,8 Man,movie,83,1992.0,0,"Action,Sci-Fi",4.9,63.0
2,12,tt0039645,1947.0,My Favorite Brunette,My Favorite Brunette,movie,87,1947.0,0,"Comedy,Crime,Film-Noir",6.8,2828.0
3,18,tt0110116,1994.0,Immortal Beloved,Immortal Beloved,movie,121,1994.0,0,"Biography,Drama,Music",7.5,22186.0
4,20,tt0078222,1972.0,Seeta Aur Geeta,Seeta Aur Geeta,movie,162,1972.0,0,"Comedy,Drama,Family",6.8,1569.0


In [137]:
imdb_df_act.columns = ['tconst','Actor_0', 'Actor_1', 'Actor_2', 'Actor_3', 'Actor_4', 'Actor_5', 'Actor_6', 'Actor_7', 'Actor_8', 'Actor_9']
imdb_df_dir.columns = ['tconst','Director_0', 'Director_1', 'Director_2', 'Director_3', 'Director_4', 'Director_5']

In [138]:
imdb_df_act.head()

,tconst,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9
0,tt0003740,Italia Almirante-Manzini,Lidia Quaranta,Bartolomeo Pagano,Carolina Catena,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0004707,Charles Chaplin,Marie Dressler,Mabel Normand,Mack Swain,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0004972,Lillian Gish,Mae Marsh,Henry B. Walthall,Miriam Cooper,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0010418,Thomas Meighan,Theodore Roberts,Raymond Hatton,Robert Cain,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0011130,John Barrymore,Martha Mansfield,Brandon Hurst,Charles Lane,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Final Dataset

In [140]:
merge_df = pd.merge(merge_df, imdb_df_act, on = ['tconst'], how = "left")
merge_df = pd.merge(merge_df, imdb_df_dir, on = ['tconst'], how = "left")
merge_df.head()

,Movie_Id,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,isAdult,genres,...,Actor_6,Actor_7,Actor_8,Actor_9,Director_0,Director_1,Director_2,Director_3,Director_4,Director_5
0,3,tt0119448,1997.0,Character,Character,movie,122,1997.0,0,"Crime,Drama,Mystery",...,NaN,NaN,NaN,NaN,Mike van Diem,NaN,NaN,NaN,NaN,NaN
1,7,tt0182668,1992.0,8 Man,8 Man,movie,83,1992.0,0,"Action,Sci-Fi",...,NaN,NaN,NaN,NaN,Yasuhiro Horiuchi,NaN,NaN,NaN,NaN,NaN
2,12,tt0039645,1947.0,My Favorite Brunette,My Favorite Brunette,movie,87,1947.0,0,"Comedy,Crime,Film-Noir",...,NaN,NaN,NaN,NaN,Elliott Nugent,NaN,NaN,NaN,NaN,NaN
3,18,tt0110116,1994.0,Immortal Beloved,Immortal Beloved,movie,121,1994.0,0,"Biography,Drama,Music",...,NaN,NaN,NaN,NaN,Bernard Rose,NaN,NaN,NaN,NaN,NaN
4,20,tt0078222,1972.0,Seeta Aur Geeta,Seeta Aur Geeta,movie,162,1972.0,0,"Comedy,Drama,Family",...,NaN,NaN,NaN,NaN,Ramesh Sippy,NaN,NaN,NaN,NaN,NaN


In [142]:
merge_df.to_csv("./IMDB_Data/Title_Data_Final.csv")